# Quantum Learning Procedure

In [ ]:
import sys
from pathlib import Path
qml_path = (Path.cwd() / "../../QML").resolve()
sys.path.insert(0, str(qml_path))

from Qsun.Qkernels import *
from Qsun.Qgates import *
from Qsun.Qmeas import *
from Qsun.Qcircuit import *
from Qsun.Qwave import *
from Qsun.Qencodes import *
from Qsun.Qdata import *

import numpy as np
import matplotlib.pyplot as plt
from src.load_datasets import *
from src.kernel_evaluation import *
from tqdm import tqdm
from typing import Dict, Tuple, Callable, List
from itertools import combinations
import problexity as px

np.random.seed(1234)

ModuleNotFoundError: No module named 'load_datasets'

### Loading 9 ansatzes from Qencodes.py

In [ ]:
ENCODING_REGISTER = {
    "YZ_CX": {
        "fn": YZ_CX_encode,
        "has_params": True,
        "has_layers": True,
    },
    "HighDim": {
        "fn": HighDim_encode,
        "has_params": False,
        "has_layers": False,
    },
    "HZY_CZ": {
        "fn": HZY_CZ_encode,
        "has_params": True,
        "has_layers": True,
    },
    "Chebyshev": {
        "fn": Chebyshev_encode,
        "has_params": True,
        "has_layers": True,
    },
    "ParamZFeatureMap": {
        "fn": ParamZFeatureMap_encode,
        "has_params": True,
        "has_layers": True,
    },
    "SeparableRX": {
        "fn": SeparableRXEncoding_encode,
        "has_params": False,
        "has_layers": False,
    },
    "HardwareEfficientRx": {
        "fn": HardwareEfficientEmbeddingRx_encode,
        "has_params": False,
        "has_layers": True,
    },
    "ZFeatureMap": {
        "fn": ZFeatureMap_encode,
        "has_params": False,
        "has_layers": True,
    },
    "ZZFeatureMap": {
        "fn": ZZFeatureMap_encode,
        "has_params": False,
        "has_layers": True,
    },
}

def encode_sample(sample: np.ndarray, encoding_name: str, n_layers: int = 2, 
                  params: np.ndarray = None):
    if encoding_name not in ENCODING_REGISTER:
        raise ValueError(f"Unknown encoding: {encoding_name}")
    config = ENCODING_REGISTER[encoding_name]
    fn = config["fn"]
    if encoding_name == "YZ_CX":
        return fn(sample, params=params, n_layers=n_layers)
    elif encoding_name == "HZY_CZ":
        return fn(sample, params=params, n_layers=n_layers)
    elif encoding_name == "Chebyshev":
        return fn(sample, params=params, n_layers=n_layers)
    elif encoding_name == "ParamZFeatureMap":
        return fn(sample, params=params, n_layers=n_layers)
    elif encoding_name == "HardwareEfficientRx":
        return fn(sample, n_layers=n_layers)
    elif encoding_name == "ZFeatureMap":
        return fn(sample, n_layers=n_layers)
    elif encoding_name == "ZZFeatureMap":
        return fn(sample, n_layers=n_layers)
    elif encoding_name == "HighDim":
        return fn(sample)
    elif encoding_name == "SeparableRX":
        return fn(sample)
    else:
        raise ValueError(f"Unknown encoding: {encoding_name}")

### Quantum Embedding Kernels

In [ ]:
def kernel_matrix(X_train: np.ndarray, X_test: np.ndarray,
                                    encoding_name: str, n_layers: int = 2,
                                    params: np.ndarray = None, random_state: int = 42,
                                    verbose: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    n_train = X_train.shape[0]
    n_test = X_test.shape[0]
    encoded_train = []
    for i in range(n_train):
        state = encode_sample(X_train[i], encoding_name, n_layers, params)
        encoded_train.append(state)
    encoded_test = []
    for i in range(n_test):
        state = encode_sample(X_test[i], encoding_name, n_layers, params)
        encoded_test.append(state)
    K_train = np.zeros((n_train, n_train))
    for i in range(n_train):
        for j in range(i, n_train):
            k_ij = state_product(encoded_train[i], encoded_train[j])**2
            K_train[i, j] = k_ij
            K_train[j, i] = k_ij
    K_test = np.zeros((n_test, n_train))
    for i in range(n_test):
        for j in range(n_train):
            K_test[i, j] = state_product(encoded_test[i], encoded_train[j])**2
    
    return K_train, K_test

def total_kernels(X_train: np.ndarray, X_test: np.ndarray,
                      encoding_names: List[str] = None, n_layers: int = 2,
                      random_state: int = 42, verbose: bool = True
                      ) -> Dict[str, Tuple[np.ndarray, np.ndarray]]:
    if encoding_names is None:
        encoding_names = list(ENCODING_REGISTER.keys())
    results = {}
    for name in encoding_names:
        try:
            K_train, K_test = kernel_matrix(
                X_train, X_test, name, n_layers, 
                random_state=random_state, verbose=verbose
            )
            results[name] = (K_train, K_test)
        except Exception as e:
            if verbose:
                print(f"  Error: {e}")
            continue
    
    return results

def get_available_encodings() -> List[str]:
    return list(ENCODING_REGISTER.keys())

In [ ]:
datasets = load_datasets(max_qubits=4 ,verbose=True)
X_train, X_test, y_train, y_test = datasets["Iris"]

print(f"\nIris dataset: Train {X_train.shape}, Test {X_test.shape}")
print(f"Available encodings: {get_available_encodings()}")

print("Sample encoding: ZFeatureMap")

K_train, K_test = kernel_matrix(
        X_train, X_test, "ZFeatureMap", n_layers=2, verbose=True
    )


  Skipped: File not found: C:\Users\Dao Duy Tung\Documents\Python\newbie\QuantumLab\ML for beginner\Kernel Discovery\pima-indians-diabete.csv
  Skipped: File not found: C:\Users\Dao Duy Tung\Documents\Python\newbie\QuantumLab\ML for beginner\Kernel Discovery\BankNote_Authentication.csv
  Skipped: File not found: C:\Users\Dao Duy Tung\Documents\Python\newbie\QuantumLab\ML for beginner\Kernel Discovery\haberman.csv

Iris dataset: Train (80, 4), Test (20, 4)
Available encodings: ['YZ_CX', 'HighDim', 'HZY_CZ', 'Chebyshev', 'ParamZFeatureMap', 'SeparableRX', 'HardwareEfficientRx', 'ZFeatureMap', 'ZZFeatureMap']
Sample encoding: ZFeatureMap


In [ ]:
datasets = load_datasets(data_dir="datasets", max_qubits=4, verbose=True)
for name, (X_tr, X_te, y_tr, y_te) in datasets.items():
    print(f"\n{name} dataset: Train {X_tr.shape}, Test {X_te.shape}")



Circle dataset: Train (80, 2), Test (20, 2)

Moons dataset: Train (80, 2), Test (20, 2)

Iris dataset: Train (80, 4), Test (20, 4)

Pima dataset: Train (613, 4), Test (154, 4)

Banknote dataset: Train (1097, 4), Test (275, 4)

Haberman dataset: Train (244, 3), Test (61, 3)


### Model Execution

In [ ]:
def total_runs(dataset_name: str = "Iris", 
                                encodings: list = None,
                                n_layers: int = 2,
                                n_runs: int = 10,
                                test_size: float = 0.2,
                                random_state: int = 42,
                                verbose: bool = True):
    if encodings is None:
        encodings = get_available_encodings()
    if verbose:
        print(f"Dataset: {dataset_name}")    
    results_accumulator = {enc: {m: {"train": [], "test": []} 
                                  for m in ["SVM", "KRC"]} 
                           for enc in encodings}
    for run in tqdm(range(n_runs)):
        seed = random_state + run
        datasets = load_datasets(data_dir="datasets", random_state=seed, test_size=test_size, max_qubits=4, verbose=False)
        X_train, X_test, y_train, y_test = datasets[dataset_name]
        kernels = total_kernels(X_train, X_test, encodings, n_layers, seed, verbose=False)
        for enc_name, (K_train, K_test) in kernels.items():
            for model_name in ["SVM", "KRC"]:
                result = evaluate_kernel(
                    K_train, K_test, y_train, y_test, enc_name, model_name
                )
                results_accumulator[enc_name][model_name]["train"].append(result.train_accuracy)
                results_accumulator[enc_name][model_name]["test"].append(result.test_accuracy)
    all_results = {}
    for enc_name in encodings:
        enc_results = []
        for model_name in ["SVM", "KRC"]:
            train_scores = results_accumulator[enc_name][model_name]["train"]
            test_scores = results_accumulator[enc_name][model_name]["test"]
            enc_results.append(KernelEvaluation(
                model_name=model_name,
                encoding_name=enc_name,
                train_accuracy=np.mean(train_scores),
                test_accuracy=np.mean(test_scores),
                train_std=np.std(train_scores),
                test_std=np.std(test_scores)
            ))
        all_results[enc_name] = enc_results
    
    return {"results": all_results}

In [ ]:
def summary(all_results):
    print(f"{'Encoding':<22} {'Model':<6} {'Train':<18} {'Test':<18}")
    print("-" * 75)
    
    best_test_acc = 0
    best_config = None
    
    for encoding_name, results in all_results.items():
        for r in results:
            train_str = f"{r.train_accuracy:.4f} ± {r.train_std:.4f}"
            test_str = f"{r.test_accuracy:.4f} ± {r.test_std:.4f}"
            print(f"{r.encoding_name:<22} {r.model_name:<6} {train_str:<18} {test_str:<18}")
            if r.test_accuracy > best_test_acc:
                best_test_acc = r.test_accuracy
                best_config = r
    
    print("-" * 75)
    print(f"Best: {best_config.encoding_name} + {best_config.model_name} = {best_test_acc:.4f} ± {best_config.test_std:.4f}")

## Accuracy Visualization

In [ ]:
def plot_accuracy(all_results):
    encodings = list(all_results.keys())
    models = ["SVM", "KRC"]
    
    test_accs = {m: [] for m in models}
    test_stds = {m: [] for m in models}
    
    for enc in encodings:
        for r in all_results[enc]:
            test_accs[r.model_name].append(r.test_accuracy)
            test_stds[r.model_name].append(r.test_std)
    
    x = np.arange(len(encodings))
    width = 0.25
    
    fig, ax = plt.subplots(figsize=(12, 5))
    
    for i, model in enumerate(models):
        ax.bar(x + i*width, test_accs[model], width, 
               yerr=test_stds[model], label=model, capsize=3)
    
    ax.set_ylabel('Test Accuracy')
    ax.set_xticks(x + width)
    ax.set_xticklabels(encodings, rotation=45, ha='right')
    ax.legend()
    ax.set_ylim([0, 1.05])
    
    plt.tight_layout()
    plt.show()


In [ ]:
import pandas as pd

def create_summary_table(all_dataset_results: Dict[str, Dict], 
                         model_name: str = "SVM") -> pd.DataFrame:
    encodings = list(ENCODING_REGISTER.keys())
    
    table_data = []
    for dataset_name, result_dict in all_dataset_results.items():
        row = {"Dataset": dataset_name}
        results = result_dict["results"]
        
        for enc_name in encodings:
            if enc_name in results:
                for r in results[enc_name]:
                    if r.model_name == model_name:
                        row[enc_name] = f"{r.test_accuracy:.4f}"
                        break
            else:
                row[enc_name] = "-"
        
        table_data.append(row)
    
    df = pd.DataFrame(table_data)
    df = df.set_index("Dataset")
    
    return df

In [ ]:
all_results = {}

for dataset_name in ["Circle", "Moons", "Iris", "Pima", "Banknote", "Haberman"]:
    print('-'*60)
    
    result = total_runs(
        dataset_name=dataset_name,
        n_layers=2,
        n_runs=10,
        random_state=42,
        verbose=True
    )
    all_results[dataset_name] = result

print("\n" + "-"*80)
print("SUMMARY TABLE (Test Accuracy - SVM)")
print("-"*80)
df_svm = create_summary_table(all_results, model_name="SVM")
print(df_svm.to_string())

print("\n" + "-"*80)
print("SUMMARY TABLE (Test Accuracy - KRC)")
print("-"*80)
df_krc = create_summary_table(all_results, model_name="KRC")
print(df_krc.to_string())

------------------------------------------------------------
Dataset: Circle


100%|██████████| 10/10 [00:01<00:00,  9.25it/s]


------------------------------------------------------------
Dataset: Moons


100%|██████████| 10/10 [00:01<00:00,  9.48it/s]


------------------------------------------------------------
Dataset: Iris


100%|██████████| 10/10 [00:02<00:00,  3.81it/s]


------------------------------------------------------------
Dataset: Pima


100%|██████████| 10/10 [01:04<00:00,  6.44s/it]


------------------------------------------------------------
Dataset: Banknote


 30%|███       | 3/10 [00:53<02:00, 17.14s/it]

In [ ]:
def label_dataframe(all_dataset_results: Dict[str, Dict], 
                          model_name: str = "SVM") -> pd.DataFrame:

    encodings = list(ENCODING_REGISTER.keys())
    
    table_data = []
    for dataset_name, result_dict in all_dataset_results.items():
        results = result_dict["results"]
        
        best_acc = 0
        best_kernel = None
        
        for enc_name in encodings:
            if enc_name in results:
                for r in results[enc_name]:
                    if r.model_name == model_name:
                        if r.test_accuracy > best_acc:
                            best_acc = r.test_accuracy
                            best_kernel = enc_name
                        break
        
        table_data.append({
            "Dataset": dataset_name,
            "Best_Kernel": best_kernel,
            "Test Accuracy": best_acc
        })
    
    df = pd.DataFrame(table_data)

    return df

df_best = label_dataframe(all_results, model_name="SVM")
df_best.style.hide(axis='index')

Dataset,Best_Kernel,Test Accuracy
Circle,ZFeatureMap,0.990000
Moons,HighDim,0.970000
Iris,HighDim,1.000000
Pima,ZFeatureMap,0.754545
Banknote,HardwareEfficientRx,1.000000
Haberman,ZFeatureMap,0.750820
